## Instalando Bibliotecas

In [ ]:
pip install openmeteo-requests requests-cache retry-requests

## Imports
Realizando os imports necessários para usar as API's

In [ ]:
import json
import requests
import pandas as pd
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

## Questão 1:
Quantos feriados há no Brasil em todo o ano de 2024?

Usei a API Nager.DATE ( Public Holidays )<br>
Aqui usei a API e laço for para contabilizar a quantidade de feriados no ano de 2024

In [ ]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)
feriados_2024 = 0

for public_holiday in public_holidays:
  feriados_2024 += 1
  
print(feriados_2024)

Resposta: Em 2024 teremos 14 feriados

## Questão 2:
Qual mês de 2024 tem o maior número de feriados?

Nessa primeira parte eu decidi criar um DataFrame com os feriados, para que eu pudesse ir utilizando<br>
essa base de dados durante as questões.

In [ ]:
df_feriados = pd.DataFrame(columns=['Holiday Name', 'Date'])
rows = []

for public_holiday in public_holidays:
  rows.append({'Holiday Name': public_holiday['name'], 'Date': public_holiday['date'],})
  
df_feriados = pd.concat([df_feriados, pd.DataFrame(rows)], ignore_index=True)
df_feriados['Date'] = pd.to_datetime(df_feriados['Date']) + pd.DateOffset(hours=3)
df_feriados['Month'] = df_feriados['Date'].dt.month_name()
df_feriados['Day'] = df_feriados['Date'].dt.weekday
df_feriados['Weekday'] = df_feriados['Date'].dt.day_name()

Visualização do DataFrame

In [ ]:
df_feriados.head(30)

Aqui onde de fato eu fiz a captura dos meses que tinham mais feriados no ano de 2024.<br>
Usei dt.month para capturar apenas o mês da coluna Date, que foi passada para datetime anteriormente<br>
e como havia meses com a mesma quantidade de feriados, usei .max para pegar o valor máximo e printar<br>
todos que estavam no valor máximo

In [ ]:
feriados_por_mes = df_feriados['Date'].dt.month.value_counts().sort_index()
max_feriados = feriados_por_mes.max()
meses_com_max_feriados = feriados_por_mes[feriados_por_mes == max_feriados]

print("Meses com o maior número de feriados:\n")
print(meses_com_max_feriados)

Resposta: <br>
2 - Fevereiro ----- 2 feriados<br>
3 - Março ----- 2 feriados <br>
5 - Maio ----- 2 feriados<br>
11 - Novembro ----- 2 feriados<br>

## Questão 3: 
Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?


Aqui eu aproveitei o DataFrame e fiz um laço for para percorrer a coluna dias, e verificar<br>
quantos dias ficam entre segunda e sexta (entre 0 e 4).

In [ ]:
count_dias_semana = 0

for dia in df_feriados['Day']:
    if 0 <= dia <= 4:
        count_dias_semana += 1

In [ ]:
print(count_dias_semana)

Resposta: 9 feriados

## Questão 4:
Qual foi a temperatura média em cada mês?

Através do site: [Open Meteo](https://open-meteo.com/en/docs/historical-weather-api#latitude=-22.9064&longitude=-43.1822&start_date=2024-01-01&end_date=2024-08-01&hourly=&daily=temperature_2m_mean&timezone=America%2FSao_Paulo) <br>
Eu consegui achar como realizar as configurar para trabalhar com a API. 

In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"daily": ["weather_code", "temperature_2m_mean"],
	"timezone": "America/Sao_Paulo"
}

responses = openmeteo.weather_api(url, params=params)

response = responses[0]
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

daily_data = {"Date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = False),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = False),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_dataframe = pd.DataFrame(data = daily_data)

daily_data["weather_code"] = daily_weather_code
daily_data['Ano'] = daily_dataframe['Date'].dt.year
daily_data['Mês'] = daily_dataframe['Date'].dt.month
daily_data["Temperatura Média"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data = daily_data)


daily_dataframe.head(50)

Continuando a resolução, aqui eu criei um DataFrame para as médias mensais

Peguei as informações do outro DataFrame e agrupei os dados usando o mês como métrica, calculando a média de temperatura para cada mês

In [ ]:
df_medias_mensais = pd.DataFrame(columns=['Mês', 'Ano', 'Temperatura Média'])

df_medias_mensais = daily_dataframe.groupby(['Ano', 'Mês'])['Temperatura Média'].mean().reset_index()

df_medias_mensais.head(10)

Resposta: <br>

Janeiro: 26.69 ºC<br>
Fevereiro: 27.21 ºC<br>
Março: 26.41 ºC<br>
Abril: 25.16 ºC<br>
Maio: 24.99 ºC<br>
Junho: 22.64 ºC<br>
Julho: 21.10 ºC<br>
Agosto: 21,22 ºC<br>

## Questão 5:
Qual foi o tempo predominante em cada mês nesse período?

Aqui eu fiz uma função utilizando o método mode() para retornar o weather_code mais recorrente<br>
e agrupei usando Ano e Mês como filtro, aplicando a função em função do weather_code<br>
e fiz o merge do dataframe medias mensais e o novo dataframe criado

In [ ]:
def weather_code_recorrente(valores):
    frequencia = valores.mode()
    return frequencia.iloc[0] if not frequencia.empty else None

weather_code_comum = daily_dataframe.groupby(['Ano', 'Mês'])['weather_code'].apply(weather_code_recorrente).reset_index()
weather_code_comum.rename(columns={'weather_code': 'Weather_Code_Mais_Recorrente'}, inplace=True)


df_resultado = pd.merge(df_medias_mensais, weather_code_comum, on=['Ano', 'Mês'])

df_resultado.head(10)

Aqui eu "carreguei" o JSON ao nosso código e fiz uma função, para fazer o mapping<br>
do weather_code do meu DataFrame com os valores que estão no JSON.

In [53]:
with open('descriptions.json', 'r') as file:
    weather_code_dict = json.load(file)

def pegar_Descrição_JSON(code, time_of_day='day'):
    if pd.isna(code):  # Verifica se o código é NaN
        return 'Descrição não disponível'
    code_str = str(int(code))
    if code_str in weather_code_dict:
        return weather_code_dict[code_str][time_of_day]['description']
    else:
        return 'Descrição não disponível'


df_resultado['Descrição do Tempo'] = df_resultado['Weather_Code_Mais_Recorrente'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))

df_resultado.head(10)

,Ano,Mês,Temperatura Média,Weather_Code_Mais_Recorrente,Descrição do Tempo
0,2024,1,26.694233,63.0,Rain
1,2024,2,27.212988,51.0,Light Drizzle
2,2024,3,26.416479,51.0,Light Drizzle
3,2024,4,25.169930,3.0,Cloudy
4,2024,5,24.990740,51.0,Light Drizzle
5,2024,6,22.648958,1.0,Mainly Sunny
6,2024,7,21.105591,3.0,Cloudy
7,2024,8,21.225414,2.0,Partly Cloudy


Resposta: Foi tomado como base para todos os dias, a parte da manhã<br>
<br>
Janeiro - Rain<br>
Fevereiro - Light Drizzle<br>
Março - Light Drizzle<br>
Abril - Cloudy<br>
Maio - Light Drizzle<br>
Junho - Mainly Sunny<br>
Julho - Cloudy<br>
Agosto - Partly Cloudy<br>

## Questão 6:
Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

Aqui eu criei um DataFrame filtrado dos feriados a partir da data escolhida.

In [54]:
data_inicio = '2024-01-01'
data_fim = '2024-08-01'

df_feriados_filtro = df_feriados[(df_feriados['Date'] >= data_inicio) & (df_feriados['Date'] <= data_fim)]
df_feriados_filtro.head(10)


,Holiday Name,Date,Month,Day,Weekday
0,New Year's Day,2024-01-01 03:00:00,January,0,Monday
1,Carnival,2024-02-12 03:00:00,February,0,Monday
2,Carnival,2024-02-13 03:00:00,February,1,Tuesday
3,Good Friday,2024-03-29 03:00:00,March,4,Friday
4,Easter Sunday,2024-03-31 03:00:00,March,6,Sunday
5,Tiradentes,2024-04-21 03:00:00,April,6,Sunday
6,Labour Day,2024-05-01 03:00:00,May,2,Wednesday
7,Corpus Christi,2024-05-30 03:00:00,May,3,Thursday
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,July,1,Tuesday


Aqui eu fiz o merge do DataFrame filtrado com as informações que eu queria, que são:<br>
Temperatura Média e weather_code, fiz o merge através das Datas. Precisei colocar a data<br>
num formato especifico, porque não estava conseguindo fazer as referências, e eu não podia<br>
tirar o timezone da Data das temperaturas, porque alterava o resultado das temperaturas.

In [55]:
df_resultado_feriados = pd.merge(df_feriados_filtro, daily_dataframe[['Date', 'Temperatura Média', 'weather_code']], on='Date', how='left')

Adicionei as Descrições do tempo

In [56]:
df_resultado_feriados['Descrição do Tempo'] = df_resultado_feriados['weather_code'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))

In [57]:
df_resultado_feriados.tail(10)

,Holiday Name,Date,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
0,New Year's Day,2024-01-01 03:00:00,January,0,Monday,24.854582,51.0,Light Drizzle
1,Carnival,2024-02-12 03:00:00,February,0,Monday,30.156668,1.0,Mainly Sunny
2,Carnival,2024-02-13 03:00:00,February,1,Tuesday,30.531670,51.0,Light Drizzle
3,Good Friday,2024-03-29 03:00:00,March,4,Friday,25.044168,61.0,Light Rain
4,Easter Sunday,2024-03-31 03:00:00,March,6,Sunday,24.531664,53.0,Drizzle
5,Tiradentes,2024-04-21 03:00:00,April,6,Sunday,23.087919,2.0,Partly Cloudy
6,Labour Day,2024-05-01 03:00:00,May,2,Wednesday,28.108751,0.0,Sunny
7,Corpus Christi,2024-05-30 03:00:00,May,3,Thursday,20.779579,51.0,Light Drizzle
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,July,1,Tuesday,21.487915,53.0,Drizzle


Resposta: <br>
Ano Novo : 24.84 ºC e Light Drizzle<br>
Carnaval (2024-02-12) : 30.15 ºC e Mainly Sunny<br>
Carnaval (2024-02-13) : 30.53 ºC e Light Drizzle<br>
Sexta-feira Santa: 25.04 ºC e Light Rain<br>
Páscoa: 24.53 ºC e Drizzle<br>
Tiradentes: 23.08 ºC e Partly Cloudy<br>
Dia do Trabalho: 28.10 ºC e Sunny<br>
Corpus Christi: 20.77 ºC e Light Drizzle<br>
Revolução Constitucionalista de 1932: 21.48 ºC e Drizzle<br>